In [1]:
#imports
from __future__ import print_function
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from mpl_toolkits.mplot3d import Axes3D
from pyspark.sql.functions import col
from functools import reduce
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import col, split
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import pandas as pd
import glob
import re
import pathlib
import json
import datetime
import numpy as np
import time
import os

In [2]:
#paths
DATASET_PATH='/Users/Soroush/Desktop/Thesis/Code/dataset/'

#from google.colab import drive
#drive.mount('/gdrive')

# **Install spark**

In [0]:
# Install spark and dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.its.dal.ca/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz #or any other version
!tar xvf spark-2.4.5-bin-hadoop2.7.tgz #based on version
!pip install -q findspark

# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7" #based on version

#imports
import findspark
findspark.init()
import pyspark # Call this only after findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#test
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
file_loc = './sample_data/california_housing_train.csv'
df = spark.read.csv(file_loc, inferSchema=True, header =True)
df.show()

# **Load Dataset**

In [3]:
#load and save .read_pickle() and .to_pickle() 

#save

#dataset.to_pickle(DATASET_PATH+"dataset.pkl")
#aggregated_dataset.to_pickle(DATASET_PATH+"aggregated_dataset.pkl")
#json_dataset.to_pickle(DATASET_PATH+"json_dataset.pkl")
#dataset.to_csv(DATASET_PATH+"dataset.csv")
#aggregated_dataset.to_csv(DATASET_PATH+"aggregated_dataset.csv")
#json_dataset.to_csv(DATASET_PATH+"json_dataset.csv")
#aggregated_dataset_rowBased.to_csv(DATASET_PATH+"aggregated_dataset_rowBased.csv")


#load

dataset=pd.read_pickle(DATASET_PATH+"dataset.pkl")
aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
json_dataset=pd.read_pickle(DATASET_PATH+"json_dataset.pkl")

In [3]:
#load dataset

# dataset address: http://traces.cs.umass.edu/index.php/Smart/Smart

#Extract File
#import tarfile
#!tar -xf '/gdrive/My Drive/a.gzip' -C '/gdrive/My Drive/'

#load

def load_smart_star_dataset(dataset_path): 
    
    #length of file path
    LENGTH=len(dataset_path)+5

    df_merged = pd.DataFrame(columns=['date'])
    
    # 2014
    #path_2014 = r'/gdrive/My Drive/Dataset/apartment/2014'
    path_2014= dataset_path+'2014'
    all_2014_paths = glob.glob(path_2014 + "/Apt*.csv")
    df_merged_2014 = pd.DataFrame(columns=['date'])
    for file_name in all_2014_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2014/", "").replace("_2014.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2014 = pd.merge(df_merged_2014, df, on='date', how='outer', left_index=True, right_index=True)
    
    # 2015
    df_merged_2015 = pd.DataFrame(columns=['date'])
    #path_2015 = r'/gdrive/My Drive/Dataset/apartment/2015'
    path_2015= dataset_path+'2015'
    all_2015_paths = glob.glob(path_2015 + "/Apt*.csv")
    for file_name in all_2015_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2015/", "").replace("_2015.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2015 = pd.merge(df_merged_2015, df, on='date', how='outer', left_index=True, right_index=True)
    
    # 2016
    df_merged_2016 = pd.DataFrame(columns=['date'])
    #path_2016 = r'/gdrive/My Drive/Dataset/apartment/2016'
    path_2016= dataset_path+'2016'
    all_2016_paths = glob.glob(path_2016 + "/Apt*.csv")
    for file_name in all_2016_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2016/", "").replace("_2016.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2016 = pd.merge(df_merged_2016, df, on='date', how='outer', left_index=True, right_index=True)
    
    # merge all years
    df_merged = df_merged_2014.append(df_merged_2015, ignore_index=True).append(df_merged_2016, ignore_index=True)
    final = df_merged
    #save
    return final

# **Preprocessing**

In [4]:
#aggregate data

def agg_by_date(final):
    final['date'] = pd.to_datetime(final['date'],)
    final.index=final['date']

    final_agg_by_hour = final.resample('60T').sum(min_count=1)  #for more than to NaN : .apply(lambda x: x.sum() if x.isnull().sum() <= 2 else np.nan)

    final = final_agg_by_hour.resample('D').aggregate(lambda x: x.tolist()) # for tuple: .aggregate(lambda x: tuple(x))

    #remove first and last row
    #aggregated_dataset.drop(pd.to_datetime('2016-12-28'),inplace=True)
    #aggregated_dataset.drop(pd.to_datetime('2014-10-15'),inplace=True)
    #final=final.iloc[1:-1]

    return final

In [5]:
#create json data from stored dataframe

def to_json(final):
    PERIOD=60

    data_for_json=final.loc[:, final.columns != 'date']

    def date_to_str(o):
        if isinstance(o, datetime.datetime):
            return o.__str__()

    json_dataframe = pd.DataFrame(columns=['data']) #creates a new dataframe that's empty
    L  = []

    import json
    import datetime
    import time

    r, c = data_for_json.shape
    for i in range(0, r):
        for j in range(0, c):
            data = {}
            data['id'] = data_for_json.columns.values[j]
            data['power'] = data_for_json.iloc[i][j]
            data['date']=data_for_json.index.tolist()[i]
            json_data = json.dumps(data,default=date_to_str)
            L.append(json_data)
            #json_dataframe=json_dataframe.append(json_data,ignore_index=True)

    json_dataframe = pd.DataFrame(L, columns=['data'])
    return json_dataframe

In [35]:
#convert column_based df to row_based

def col_to_row(df):
    #house_id=df.columns
    df.reset_index(inplace=True)
    return pd.melt(df, id_vars=['date'],value_name='power')

#aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
#aggregated_dataset_rowBased=col_to_row(aggregated_dataset)
#aggregated_dataset_rowBased.sort_values(['date', 'variable'], ascending=[True, True],inplace=True)

# **Model**

In [99]:
#create SparkSession
spark = SparkSession.builder.appName("anomaly_detection").master("local[2]").getOrCreate()

#define schema
schema = StructType([
    StructField("num", IntegerType()),
    StructField("date", TimestampType()),
    StructField("id", StringType()),
    StructField("power", StringType())])

#read data
sdf = spark.read.format('csv').options(header='true', schema=schema).load(DATASET_PATH+"aggregated_dataset_rowBased.csv")

#convert power to array
def string_power_to_array(sdf):
    temp=sdf.withColumn("power", f.regexp_replace(f.regexp_replace(f.col("power"), "\\[", ""),"\\]","")
                        .alias("power"))
    temp=temp.withColumn("power",split(col("power"), ",\s*")
                        .cast(ArrayType(FloatType())).alias("power"))
    return temp

sdf=string_power_to_array(sdf)
sdf.printSchema()

root
 |-- num: string (nullable = true)
 |-- date: string (nullable = true)
 |-- variable: string (nullable = true)
 |-- power: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [ ]:
# Define udf
from pyspark.sql.functions import udf

def string_to_list(t: str):
    return t.strip('][').split(', ') 


udf_parse_json = udf(lambda str: string_to_list(str), json_schema)

# **MAIN**

In [ ]:
#load dataset
dataset = load_smart_star_dataset(DATASET_PATH)

#aggregate by date
aggregated_dataset=agg_by_date(dataset)
aggregated_dataset=aggregated_dataset[1:-1]

#to json
json_dataset=to_json(aggregated_dataset)

# **TEST**

In [24]:
#sdf.printSchema()

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

#cast
def string_to_list(t: str):
    return t.strip('][').split(', ') 
     

sdf.withColumn("greetings", string_to_list(col("power"))).show()

root
 |-- _c0: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- variable: string (nullable = true)
 |-- power: string (nullable = true)



In [56]:
from pyspark.sql.types import *

schema = StructType([
  StructField("num", IntegerType()),
    StructField("date", TimestampType()),
    StructField("id", StringType()),
  StructField("power", ArrayType(
      StructType([
          StructField("H0", FloatType(), True),
          StructField("H1", FloatType(), True),
          StructField("H2", FloatType(), True),
          StructField("H3", FloatType(), True),
          StructField("H4", FloatType(), True),
          StructField("H5", FloatType(), True),
          StructField("H6", FloatType(), True),
          StructField("H7", FloatType(), True),
          StructField("H8", FloatType(), True),
          StructField("H9", FloatType(), True),
          StructField("H10", FloatType(), True),
          StructField("H11", FloatType(), True),
          StructField("H12", FloatType(), True),
          StructField("H13", FloatType(), True),
          StructField("H14", FloatType(), True),
          StructField("H15", FloatType(), True),
          StructField("H16", FloatType(), True),
          StructField("H17", FloatType(), True),
          StructField("H18", FloatType(), True),
          StructField("H19", FloatType(), True),
          StructField("H20", FloatType(), True),
          StructField("H21", FloatType(), True),
          StructField("H22", FloatType(), True),
          StructField("H23", FloatType(), True)
      ])
   )
             )])

a = spark.read.format('csv').schema(schema).option("header", "true").load(DATASET_PATH+"f.csv")
a.show()

AnalysisException: 'CSV data source does not support array<struct<H0:float,H1:float,H2:float,H3:float,H4:float,H5:float,H6:float,H7:float,H8:float,H9:float,H10:float,H11:float,H12:float,H13:float,H14:float,H15:float,H16:float,H17:float,H18:float,H19:float,H20:float,H21:float,H22:float,H23:float>> data type.;'

In [221]:
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

spark = SparkSession.builder.appName("anomaly_detection").master("local[2]").getOrCreate()

#load dataset
sdf = spark.read.format('csv').options(header='true', inferSchema='true').load(DATASET_PATH+"aggregated_dataset_rowBased.csv")

#cast
def string_to_list(t):
    return t.strip('][').split(', ') 

sdf.show()
#sdf.take(10)

+-----+-------------------+--------+--------------------+
|  _c0|               date|variable|               power|
+-----+-------------------+--------+--------------------+
|15624|2014-08-14 00:00:00|    Apt1|[0.0, 0.0, 0.0, 0...|
|85932|2014-08-14 00:00:00|   Apt10|[0.0, 0.0, 0.0, 0...|
|78120|2014-08-14 00:00:00|  Apt100|[0.0, 0.0, 0.0, 0...|
|78988|2014-08-14 00:00:00|  Apt101|[0.0, 0.0, 0.0, 0...|
|37324|2014-08-14 00:00:00|  Apt102|[0.985781111, 0.5...|
|36456|2014-08-14 00:00:00|  Apt103|[0.0, 0.0, 0.0, 0...|
| 3472|2014-08-14 00:00:00|  Apt104|[0.0, 0.0, 0.0, 0...|
| 4340|2014-08-14 00:00:00|  Apt105|[1.42481, 2.96944...|
|68572|2014-08-14 00:00:00|  Apt106|[0.00015000000000...|
|67704|2014-08-14 00:00:00|  Apt107|[0.0, 0.0, 0.0, 0...|
|42532|2014-08-14 00:00:00|  Apt108|[0.0, 0.0, 0.0, 0...|
|39928|2014-08-14 00:00:00|  Apt109|[0.0, 0.0, 0.0, 0...|
|85064|2014-08-14 00:00:00|   Apt11|[0.0, 0.0, 0.0, 0...|
|52948|2014-08-14 00:00:00|  Apt110|[0.0, 0.0, 0.0, 0...|
|53816|2014-08

In [ ]:
#session = SparkSession.builder.appName("StackOverFlowSurvey").getOrCreate()
type(house_id)
res = {house_id[i]: 'int' for i in range(len(house_id))} 
res

In [36]:
aggregated_dataset_rowBased=col_to_row(aggregated_dataset)
aggregated_dataset_rowBased.to_csv(DATASET_PATH+"aggregated_dataset_rowBased.csv")

In [51]:
from pyspark.mllib.linalg import Vectors
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1])]).toDF(['features'])

In [ ]:
sdf = sdf.select(*(col(c).cast("float").alias(c) for c in house_id))
sdf.take(10)

In [ ]:
test=aggregated_dataset.iloc[1].to_frame()
test=test.reset_index()
test.columns=['id','features']
test.head()

test['features'].values.tolist()

temp = pd.DataFrame(test['features'].values.tolist(), columns=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])
temp.head()
